실행 위치: Google Colab

# 1. 모델 로드
 - AutoModelForCausalLM이 아닌 AutoModelForSequenceClassification 로 로드
 - AutoModelForSequenceClassification 라이브러리는 입력된 문장을 기반으로 분류를 수행하는 모델을 로드할 때 사용

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

model_name = "openai-community/gpt2-xl"
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForSequenceClassification.from_config(
    config,
    trust_remote_code=True
).to('cuda')


# 2. 모델 구조 보기

In [ ]:
print(model)

- score의 out_features = 2이며 최종 2차원의 벡터값으로 리턴

In [ ]:
print(f'score Layer의 shape: {model.score.weight.shape}')
print('---------------------------')
print(model.score.weight)

# 3. 추론 수행

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

In [ ]:
input = "The core of the Transformer model is attention."
encoded_input = tokenizer(
    input,
    return_tensors="pt"
).to('cuda')

print(encoded_input)
print('-------------')
token_str = tokenizer.convert_ids_to_tokens(encoded_input['input_ids'][0])
for token in zip(encoded_input['input_ids'][0], token_str):
    print(token)

In [ ]:
outputs = model(**encoded_input)

print(outputs.logits)
print('-------------')

import torch
print(torch.softmax(outputs.logits, dim=-1))

- gpt2-xl 모델은 Causal LM 용도로 학습되었으며 Sentence Classification 용도로는 학습되지 않음
- 위 추론은 동작은 하지만 아무 의미없는 값임
- 중요한 것은 Encoder 모델 = 분류, Decoder 모델 = 토큰생성 이라는 것은 아니라는 것이며
- Encoder 모델로도 토큰 생성이 가능하고, Decoder 모델로 분류도 가능하다는 것임 
- 그렇다면 Encoder, Decoder를 구분하는 것인 무엇인지에 대한 이해 필요